In [1]:
# Make sure that we use SageMaker 1.x
!pip install sagemaker==1.72.0

     |████████████████████████████████| 297 kB 21.9 MB/s eta 0:00:01
  Created wheel for sagemaker: filename=sagemaker-1.72.0-py2.py3-none-any.whl size=388346 sha256=0bfb0fc5e6d887f8f8e60d5b2d91bbd58641f5f12fd994fae2fd041f1eb91d45
  Stored in directory: /home/ec2-user/.cache/pip/wheels/c3/58/70/85faf4437568bfaa4c419937569ba1fe54d44c5db42406bbd7
Successfully built sagemaker
  Attempting uninstall: smdebug-rulesconfig
    Found existing installation: smdebug-rulesconfig 1.0.1
    Uninstalling smdebug-rulesconfig-1.0.1:
      Successfully uninstalled smdebug-rulesconfig-1.0.1
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.68.0
    Uninstalling sagemaker-2.68.0:
      Successfully uninstalled sagemaker-2.68.0
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


In [2]:
import os
import glob

def read_imdb_data(data_dir='../data_vi'):
    data = {}
    labels = {}
    
    for data_type in ['train', 'test']:
        data[data_type] = {}
        labels[data_type] = {}
        
        for sentiment in ['pos', 'neg']:
            data[data_type][sentiment] = []
            labels[data_type][sentiment] = []
            
            path = os.path.join(data_dir, data_type, sentiment, '*.txt')
            files = glob.glob(path)
            
            for f in files:
                with open(f) as review:
                    data[data_type][sentiment].append(review.read())
                    # Here we represent a positive review by '1' and a negative review by '0'
                    labels[data_type][sentiment].append(1 if sentiment == 'pos' else 0)
                    
            assert len(data[data_type][sentiment]) == len(labels[data_type][sentiment]), \
                    "{}/{} data size does not match labels size".format(data_type, sentiment)
                
    return data, labels

In [3]:
data, labels = read_imdb_data()
print("IMDB reviews: train = {} pos / {} neg, test = {} pos / {} neg".format(
            len(data['train']['pos']), len(data['train']['neg']),
            len(data['test']['pos']), len(data['test']['neg'])))

IMDB reviews: train = 15000 pos / 15000 neg, test = 5000 pos / 5000 neg


In [4]:
from sklearn.utils import shuffle

def prepare_imdb_data(data, labels):
    """Prepare training and test sets from IMDb movie reviews."""
    
    #Combine positive and negative reviews and labels
    data_train = data['train']['pos'] + data['train']['neg']
    data_test = data['test']['pos'] + data['test']['neg']
    labels_train = labels['train']['pos'] + labels['train']['neg']
    labels_test = labels['test']['pos'] + labels['test']['neg']
    
    #Shuffle reviews and corresponding labels within training and test sets
    data_train, labels_train = shuffle(data_train, labels_train)
    data_test, labels_test = shuffle(data_test, labels_test)
    
    # Return a unified training data, test data, training labels, test labets
    return data_train, data_test, labels_train, labels_test

In [5]:
train_X, test_X, train_y, test_y = prepare_imdb_data(data, labels)
print("IMDb reviews (combined): train = {}, test = {}".format(len(train_X), len(test_X)))

IMDb reviews (combined): train = 30000, test = 10000


In [6]:
print(train_X[100])
print(train_y[100])

Thức uống rất ngon , giá_cả lại bình_dân , rất thoải_mái , ở đây lại còn có cả cài phần mên IOS cho iphone và ipad ... nói_chung là very good ...

1


In [7]:
f = open("../data_vi/vietnamese-stopwords.txt", "r")
stopwords = f.read()
stopwords = stopwords.replace('\n', ' ').split(' ')
stopwords = list(set(stopwords))

f = open("../data_vi/teen_code.txt", "r")
teeencode = {}
for t in f.read().split('\n'):
    t = t.split('\t')
    teeencode[t[0]] = t[1]

In [8]:
!pip install regex

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


In [9]:
import nltk
from nltk.stem.porter import *
import string

import regex
from bs4 import BeautifulSoup

def review_to_words(review, stopwords, teencodes):
    stemmer = PorterStemmer()
    
    text = BeautifulSoup(review, "html.parser").get_text() # Remove HTML tags
    text = re.sub(r"[^a-záàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ]", " ", text.lower()) # Convert to lower case
    text = text.lower() # Convert to lower case
    text = text.strip()
    words = text.split(' ') # Split string into words
    words = [w for w in words if w not in stopwords] # Remove stopwords
    words = [w for w in words if w != '']
    new_words = []
    for w in words:
        if w in teencodes:
            new_words.append(teencodes[w])
        else:
            new_words.append(w)
#     words = [PorterStemmer().stem(w) for w in words] # stem
    
    return new_words

In [10]:
id = 6
print(train_X[id])
print(review_to_words(train_X[id], stopwords, teeencode))

1 . Decor đẹp , đi buổi_sáng nên vắng khá yên_tĩnh
2 . Giá cao
3 . Nhân_viên không thân_thiện , quán vắng te mà bắt mình đổi chỗ 2 lần với thái_độ khó_chịu như kiểu đuổi đi chỗ khác vậy . Cũng không xin_lỗi đc câu nào . Đã_vậy cái kiểu như tưởng khách không có tiền vậy . Phục_vụ lâu .
4 . Thức uống dở .

['decor', 'đẹp', 'vắng', 'yên', 'tĩnh', 'viên', 'thiện', 'quán', 'vắng', 'chỗ', 'độ', 'kiểu', 'đuổi', 'chỗ', 'lỗi', 'được', 'kiểu', 'tưởng', 'tiền', 'phục', 'vụ', 'thức', 'uống']


In [11]:
import pickle

cache_dir = os.path.join("../cache_vi", "sentiment_analysis")  # where to store cache files
os.makedirs(cache_dir, exist_ok=True)  # ensure cache directory exists

def preprocess_data(data_train, data_test, labels_train, labels_test,
                    cache_dir=cache_dir, cache_file="preprocessed_data.pkl"):
    """Convert each review to words; read from cache if available."""

    # If cache_file is not None, try to read from it first
    cache_data = None
    if cache_file is not None:
        try:
            with open(os.path.join(cache_dir, cache_file), "rb") as f:
                cache_data = pickle.load(f)
            print("Read preprocessed data from cache file:", cache_file)
        except:
            pass  # unable to read from cache, but that's okay
    
    # If cache is missing, then do the heavy lifting
    if cache_data is None:
        # Preprocess training and test data to obtain words for each review
        words_train = [review_to_words(review, stopwords, teeencode) for review in data_train]
        words_test = [review_to_words(review, stopwords, teeencode) for review in data_test]
        
        # Write to cache file for future runs
        if cache_file is not None:
            cache_data = dict(words_train=words_train, words_test=words_test,
                              labels_train=labels_train, labels_test=labels_test)
            with open(os.path.join(cache_dir, cache_file), "wb") as f:
                pickle.dump(cache_data, f)
            print("Wrote preprocessed data to cache file:", cache_file)
    else:
        # Unpack data loaded from cache file
        words_train, words_test, labels_train, labels_test = (cache_data['words_train'],
                cache_data['words_test'], cache_data['labels_train'], cache_data['labels_test'])
    
    return words_train, words_test, labels_train, labels_test

In [12]:
train_X_p, test_X_p, train_y_p, test_y_p = preprocess_data(train_X, test_X, train_y, test_y)

Wrote preprocessed data to cache file: preprocessed_data.pkl


In [13]:
import numpy as np

def build_dict(data, vocab_size = 5000):
    """Construct and return a dictionary mapping each of the most frequently appearing words to a unique integer."""
    
    word_count = {} # A dict storing the words that appear in the reviews along with how often they occur
    for d in data:
        for w in d:
            word_count[w] = word_count.get(w, 0) + 1
    # Sort the words found in `data` so that sorted_words[0] is the most frequently appearing word and
    # sorted_words[-1] is the least frequently appearing word.
    
    # sorted_words = list(dict(sorted(word_count.items(), key=lambda item: item[1], reverse=True)).items())
    sorted_words = sorted(word_count, key=word_count.get, reverse=True)
    
    word_dict = {} # This is what we are building, a dictionary that translates words into integers
    for idx, word in enumerate(sorted_words[:vocab_size - 2]): # The -2 is so that we save room for the 'no word'
        word_dict[word] = idx + 2                              # 'infrequent' labels
        
    return word_dict

In [14]:
word_dict = build_dict(train_X_p)

In [15]:
# Use this space to determine the five most frequently appearing words in the training set.
for word in list(word_dict.keys())[:5]:
    print(word)

không
quán
ngon
món
viên


In [16]:
data_dir = '../data_vi/pytorch' # The folder we will use for storing data
if not os.path.exists(data_dir): # Make sure that the folder exists
    os.makedirs(data_dir)

In [17]:
with open(os.path.join(data_dir, 'word_dict.pkl'), "wb") as f:
    pickle.dump(word_dict, f)

In [18]:
def convert_and_pad(word_dict, sentence, pad=500):
    NOWORD = 0 # We will use 0 to represent the 'no word' category
    INFREQ = 1 # and we use 1 to represent the infrequent words, i.e., words not appearing in word_dict
    
    working_sentence = [NOWORD] * pad
    
    for word_index, word in enumerate(sentence[:pad]):
        if word in word_dict:
            working_sentence[word_index] = word_dict[word]
        else:
            working_sentence[word_index] = INFREQ
            
    return working_sentence, min(len(sentence), pad)

def convert_and_pad_data(word_dict, data, pad=500):
    result = []
    lengths = []
    
    for sentence in data:
        converted, leng = convert_and_pad(word_dict, sentence, pad)
        result.append(converted)
        lengths.append(leng)
        
    return np.array(result), np.array(lengths)

In [19]:
train_X_cv, train_X_len = convert_and_pad_data(word_dict, train_X_p)
test_X_cv, test_X_len = convert_and_pad_data(word_dict, test_X_p)

In [20]:
print(train_X_cv[0])

[ 186   12 1056 2849 2849  101   86 2382 1928   31  753  275  611    7
  338  717  192  135   81  166  427  275  611    7 2179   19  434  272
  506  166  187 1185  217  226   79    7  506   67  166  297 4530  403
    7    4   13   86  395 2382 1928  226   66   12  966  101   27    2
  101  101 3882   48  101    1    1    2  101    2  101    2  101    6
   48    2  896    2   47 1560  371  516  101  101  105  150  269    4
  776    2  403 1606  809  403 1606    4  376    8  149   91  439    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [21]:
import pandas as pd
    
pd.concat([pd.DataFrame(train_y_p), pd.DataFrame(train_X_len), pd.DataFrame(train_X_cv)], axis=1) \
        .to_csv(os.path.join(data_dir, 'train.csv'), header=False, index=False)

In [22]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/sentiment_rnn_vi'

role = sagemaker.get_execution_role()

In [23]:
input_data = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)

In [24]:
import torch
import torch.utils.data

# Read in only the first 250 rows
train_sample = pd.read_csv(os.path.join(data_dir, 'train.csv'), header=None, names=None, nrows=250)

# Turn the input pandas dataframe into tensors
train_sample_y = torch.from_numpy(train_sample[[0]].values).float().squeeze()
train_sample_X = torch.from_numpy(train_sample.drop([0], axis=1).values).long()

# Build the dataset
train_sample_ds = torch.utils.data.TensorDataset(train_sample_X, train_sample_y)
# Build the dataloader
train_sample_dl = torch.utils.data.DataLoader(train_sample_ds, batch_size=50)

In [25]:
def train(model, train_loader, epochs, optimizer, loss_fn, device):
    for epoch in range(1, epochs + 1):
        model.train()
        total_loss = 0
        for batch in train_loader:         
            batch_X, batch_y = batch
            
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            
            # TODO: Complete this train method to train the model provided.
            # the gradients of all optimized variables are cleared
            optimizer.zero_grad()
            # forward pass
            output = model.forward(batch_X)
            # calculate the batch loss
            loss = loss_fn(output, batch_y)
            # backpropagation
            loss.backward()
            #optimization
            optimizer.step()
            
            total_loss += loss.data.item()
        print("Epoch: {}, BCELoss: {}".format(epoch, total_loss / len(train_loader)))

In [27]:
import torch.optim as optim
from train.model import LSTMClassifier

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMClassifier(32, 100, 5000).to(device)
optimizer = optim.Adam(model.parameters())
loss_fn = torch.nn.BCELoss()

train(model, train_sample_dl, 10, optimizer, loss_fn, device)

Epoch: 1, BCELoss: 0.69428950548172
Epoch: 2, BCELoss: 0.6860834836959839
Epoch: 3, BCELoss: 0.6793828725814819
Epoch: 4, BCELoss: 0.6721738457679749
Epoch: 5, BCELoss: 0.6636547684669495
Epoch: 6, BCELoss: 0.6528251767158508
Epoch: 7, BCELoss: 0.6382176995277404
Epoch: 8, BCELoss: 0.61777583360672
Epoch: 9, BCELoss: 0.5898136258125305
Epoch: 10, BCELoss: 0.5548046827316284


In [28]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point="train.py",
                    source_dir="train",
                    role=role,
                    framework_version='0.4.0',
                    py_version="py3",
                    train_instance_count=1,
                    train_instance_type='ml.m4.xlarge',
                    hyperparameters={
                        'epochs': 10,
                        'hidden_dim': 200,
                    })

In [29]:
estimator.fit({'training': input_data})

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2021-11-24 16:29:54 Starting - Starting the training job...
2021-11-24 16:29:58 Starting - Launching requested ML instances......
2021-11-24 16:31:06 Starting - Preparing the instances for training............
2021-11-24 16:32:58 Downloading - Downloading input data...
2021-11-24 16:33:50 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-11-24 16:33:52,082 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2021-11-24 16:33:52,084 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2021-11-24 16:33:52,097 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-11-24 16:33:53,540 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-11-24 16:33:54,058 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generatin

Epoch: 1, BCELoss: 0.6290012204040916
Epoch: 2, BCELoss: 0.5271409492371446
Epoch: 3, BCELoss: 0.4502586959782293
Epoch: 4, BCELoss: 0.40641631615363943
Epoch: 5, BCELoss: 0.37040331909212015
Epoch: 6, BCELoss: 0.3496713289770029
Epoch: 7, BCELoss: 0.3377927854909735
Epoch: 8, BCELoss: 0.3302478664002176
Epoch: 9, BCELoss: 0.30811207880408076

2021-11-24 18:37:03 Uploading - Uploading generated training model
2021-11-24 18:37:03 Completed - Training job completed
Epoch: 10, BCELoss: 0.2985565053709483
2021-11-24 18:36:55,788 sagemaker-containers INFO     Reporting training SUCCESS
Training seconds: 7445
Billable seconds: 7445


In [30]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


---------!

In [36]:
test_X = pd.concat([pd.DataFrame(test_X_len), pd.DataFrame(test_X_cv)], axis=1)

In [37]:
# We split the data into chunks and send each chunk seperately, accumulating the results.
def predict(data, rows=512):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = np.array([])
    for array in split_array:
        predictions = np.append(predictions, predictor.predict(array))
    
    return predictions

In [38]:
predictions = predict(test_X.values)
predictions = [round(num) for num in predictions]

In [39]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y_p, predictions)

0.8381

In [40]:
test_review = 'Quán cơm sườn, gà nướng đặc biệt luôn, nhìn rất ngon thịt nướng bằng tủ quây, miếng thịt nướng màu đẹp trông ngon lạ'

In [41]:
#Convert test_review into a form usable by the model and save the results in test_data
test_review_data, test_review_len = convert_and_pad_data(word_dict, review_to_words(test_review, stopwords, teeencode))

test_data = np.array(test_review_data)
test_data = np.insert(test_data, 0, test_review_len)

test_data = test_data[None, :]

In [42]:
predictor.predict(test_data)

array(0.48984012, dtype=float32)

In [43]:
estimator.delete_endpoint()

estimator.delete_endpoint() will be deprecated in SageMaker Python SDK v2. Please use the delete_endpoint() function on your predictor instead.


In [ ]:
from sagemaker.predictor import RealTimePredictor
from sagemaker.pytorch import PyTorchModel

class StringPredictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(StringPredictor, self).__init__(endpoint_name, sagemaker_session, content_type='text/plain')

model = PyTorchModel(model_data=estimator.model_data,
                     role = role,
                     framework_version='0.4.0',
                     entry_point='predict.py',
                     source_dir='serve_vi',
                     predictor_cls=StringPredictor)
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

In [ ]:
import glob

def test_reviews(data_dir='../data_vi', stop=250):
    
    results = []
    ground = []
    
    # We make sure to test both positive and negative reviews    
    for sentiment in ['pos', 'neg']:
        
        path = os.path.join(data_dir, 'test', sentiment, '*.txt')
        files = glob.glob(path)
        
        files_read = 0
        
        print('Starting ', sentiment, ' files')
        
        # Iterate through the files and send them to the predictor
        for f in files:
            with open(f) as review:
                # First, we store the ground truth (was the review positive or negative)
                if sentiment == 'pos':
                    ground.append(1)
                else:
                    ground.append(0)
                # Read in the review and convert to 'utf-8' for transmission via HTTP
                review_input = review.read().encode('utf-8')
                # Send the review to the predictor and store the results
                # results.append(float(predictor.predict(review_input)))
                results.append(int(predictor.predict(review_input)))
                
            # Sending reviews to our endpoint one at a time takes a while so we
            # only send a small number of reviews
            files_read += 1
            if files_read == stop:
                break
            
    return ground, results

In [ ]:
ground, results = test_reviews()

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(ground, results)

In [ ]:
predictor.predict(test_review)

In [ ]:
predictor.endpoint

In [ ]:
predictor.delete_endpoint()